# Experiments with backtesting.py
This notebook is to explore the `backtesting.py` to evaluate strategy:
1. Based on conventional TA (talib etc.)
2. See how can we introduce ML model as TA

Install ta-lib on Mac with brew first `brew install ta-lib` and then `pip install TA-Lib`

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import yfinance as yf
import talib 
from utils import get_stock_data
import time
import warnings

/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### 1. Standard Examples
First, use standard example; where goal is just to see how we can use the library. In the following, we will use SMA as an indicator which is already available in `backtesting.py`. We will also use stop loss and take profit to limit the exposure and to ensure risk-reward ratio.

In [2]:
class SmaCrossWithSLTP(Strategy):
    short_window = 10  # Fast SMA
    long_window = 20   # Slow SMA
    stop_loss_pct = 0.02   # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit
    
    # All the necessary indicator needs to define in the init method
    def init(self):
        self.sma_short = self.I(SMA, self.data.Close, self.short_window)
        self.sma_long = self.I(SMA, self.data.Close, self.long_window)
    
    # It defines the logic of the strategy
    def next(self):
        price = self.data.Close[-1]  # Current price
        if crossover(self.sma_short, self.sma_long):  # Buy condition
            stop_loss = price * (1 - self.stop_loss_pct)
            take_profit = price * (1 + self.take_profit_pct)
            self.buy(sl=stop_loss, tp=take_profit)  # Order with SL and TP
        
        elif crossover(self.sma_long, self.sma_short):  # Sell condition
            self.position.close()  # Close existing position

# Load sample data which comes with the library to test
df = GOOG

# Run Backtest
bt = Backtest(df, SmaCrossWithSLTP, cash=10000, commission=0.002)
stats = bt.run()
print(stats)
# bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     9.82309
Equity Final [$]                  11532.01791
Equity Peak [$]                   12667.87271
Commissions [$]                    2041.36509
Return [%]                           15.32018
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                     1.68634
Volatility (Ann.) [%]                 6.32304
CAGR [%]                              1.15945
Sharpe Ratio                           0.2667
Sortino Ratio                         0.40031
Calmar Ratio                          0.14875
Alpha [%]                            -4.51391
Beta                                  0.03266
Max. Drawdown [%]                   -11.33648
Avg. Drawdown [%]                    -3.53535
Max. Drawdown Duration      742 days 00:00:00
Avg. Drawdown Duration      172 days 00:00:00
# Trades                          

## 2a. Integrating other indicators from TA-LIB
Now, moving to `ta-lib` for more indicators. In the following, we will use RSI along with SL and TP.

In [13]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14    # RSI period
    rsi_buy = 30       # RSI oversold threshold
    rsi_sell = 70      # RSI overbought threshold
    stop_loss_pct = 0.02  # 2% stop-loss
    take_profit_pct = 0.05  # 5% take-profit

    def init(self):    
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Entry Condition: RSI below 30 & SMA crossover
        if self.rsi[-1] < self.rsi_buy :
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  # Stop-Loss 2% below entry
            tp = price * (1 + self.take_profit_pct)  # Take-Profit 5% above entry
            self.buy(sl=sl, tp=tp)  # Place order with SL & TP

        # Exit Condition: RSI above 70 OR SMA crossover in opposite direction
        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# Save HTML file for plots for in-browser visualization
# bt.plot(filename="rsi_strategy.html")

Backtest.run:   0%|          | 0/519 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=380: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=381: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                     1.31086
Equity Final [$]                  10278.12869
Equity Peak [$]                   10278.12869
Commissions [$]                      77.58101
Return [%]                            2.78129
Buy & Hold Return [%]                 56.6491
Return (Ann.) [%]                     1.30301
Volatility (Ann.) [%]                 1.55292
CAGR [%]                              0.89369
Sharpe Ratio                          0.83907
Sortino Ratio                         3.39637
Calmar Ratio                           1.3681
Alpha [%]                             2.48898
Beta                                  0.00516
Max. Drawdown [%]                    -0.95243
Avg. Drawdown [%]                    -0.95243
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                          

## 2b. Using pandas_ta  
`pandas_ta` is a simpler wrapper for dataframe. Some of the TA are build upon above-used `ta-lib` so install both.

In [14]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 30
    rsi_sell = 70
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        # Compute RSI using pandas_ta
        rsi_series = self.data.df.ta.rsi(length=self.rsi_period)
        self.rsi = self.I(lambda: rsi_series)

    def next(self):
        if self.rsi[-1] < self.rsi_buy:
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

# Assuming get_stock_data returns a DataFrame with OHLCV
df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
# bt.plot(filename="rsi_strategy.html")


Backtest.run:   0%|          | 0/519 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=380: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=381: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                     

Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                     1.31086
Equity Final [$]                  10278.12869
Equity Peak [$]                   10278.12869
Commissions [$]                      77.58101
Return [%]                            2.78129
Buy & Hold Return [%]                 56.6491
Return (Ann.) [%]                     1.30301
Volatility (Ann.) [%]                 1.55292
CAGR [%]                              0.89369
Sharpe Ratio                          0.83907
Sortino Ratio                         3.39637
Calmar Ratio                           1.3681
Alpha [%]                             2.48898
Beta                                  0.00516
Max. Drawdown [%]                    -0.95243
Avg. Drawdown [%]                    -0.95243
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                          

#### More TA based strategigies (Bollinger band)

In [15]:
class RsiBollingerStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 30
    rsi_sell = 70
    bb_period = 20
    bb_std = 2
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        df = self.data.df

        # Indicators from pandas_ta
        self.rsi = self.I(lambda: df.ta.rsi(length=self.rsi_period))
        bb = df.ta.bbands(length=self.bb_period, std=self.bb_std)

        # Use lower & upper bands from BB
        self.bb_lower = self.I(lambda: bb['BBL_20_2.0'])
        self.bb_upper = self.I(lambda: bb['BBU_20_2.0'])

    def next(self):
        price = self.data.Close[-1]

        # Buy condition: RSI < 30 and price below lower BB
        if self.rsi[-1] < self.rsi_buy and price < self.bb_lower[-1]:
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        # Sell condition: RSI > 70 or price > upper BB
        elif self.position:
            if self.rsi[-1] > self.rsi_sell or price > self.bb_upper[-1]:
                self.position.close()

# Fetch data
df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run backtest
bt = Backtest(df, RsiBollingerStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
# bt.plot(filename="rsi_bb_strategy.html")


Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                      1.1236
Equity Final [$]                  10330.02134
Equity Peak [$]                   10330.02134
Commissions [$]                      40.50959
Return [%]                            3.30021
Buy & Hold Return [%]                59.98219
Return (Ann.) [%]                     1.54406
Volatility (Ann.) [%]                 1.58944
CAGR [%]                              1.05862
Sharpe Ratio                          0.97145
Sortino Ratio                         5.38308
Calmar Ratio                          2.62117
Alpha [%]                             3.05516
Beta                                  0.00409
Max. Drawdown [%]                    -0.58907
Avg. Drawdown [%]                    -0.58907
Max. Drawdown Duration        3 days 00:00:00
Avg. Drawdown Duration        3 days 00:00:00
# Trades                          

In [16]:
class ImprovedRsiBbStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 40
    rsi_sell = 60
    bb_period = 20
    bb_std = 2
    ema_period = 200
    atr_period = 20
    risk_factor = 1.5  # Multiplier for ATR for SL/TP

    def init(self):
        df = self.data.df
        self.rsi = self.I(lambda: df.ta.rsi(length=self.rsi_period))
        bb = df.ta.bbands(length=self.bb_period, std=self.bb_std)
        self.bb_lower = self.I(lambda: bb['BBL_20_2.0'])
        self.bb_upper = self.I(lambda: bb['BBU_20_2.0'])
        self.ema = self.I(lambda: df.ta.ema(length=self.ema_period))
        self.atr = self.I(lambda: df.ta.atr(length=self.atr_period))

    def next(self):
        price = self.data.Close[-1]

        # Long Entry
        if not self.position:
            if price > self.ema[-1]:  # Trend filter: Only long above EMA
                if self.rsi[-1] < self.rsi_buy and price < self.bb_lower[-1]:
                    sl = price - self.atr[-1] * self.risk_factor
                    tp = price + self.atr[-1] * self.risk_factor * 2
                    self.buy(sl=sl, tp=tp)

            # Short Entry
            elif price < self.ema[-1]:
                if self.rsi[-1] > self.rsi_sell and price > self.bb_upper[-1]:
                    sl = price + self.atr[-1] * self.risk_factor
                    tp = price - self.atr[-1] * self.risk_factor * 2
                    self.sell(sl=sl, tp=tp)

        else:
            # Optional: Add exit logic for open positions if SL/TP not hit
            if self.position.is_long and (self.rsi[-1] > 60 or price > self.bb_upper[-1]):
                self.position.close()

            elif self.position.is_short and (self.rsi[-1] < 40 or price < self.bb_lower[-1]):
                self.position.close()


df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")
# Backtest it
bt = Backtest(df, ImprovedRsiBbStrategy, cash=10000, commission=0.002)
stats = bt.run()
print(stats)
#bt.plot()


Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                     7.86517
Equity Final [$]                  10779.75686
Equity Peak [$]                   10779.75686
Commissions [$]                     197.13103
Return [%]                            7.79757
Buy & Hold Return [%]                 5.90621
Return (Ann.) [%]                     3.60686
Volatility (Ann.) [%]                 6.32538
CAGR [%]                              2.46508
Sharpe Ratio                          0.57022
Sortino Ratio                         0.89976
Calmar Ratio                          0.94302
Alpha [%]                             7.40682
Beta                                  0.06616
Max. Drawdown [%]                    -3.82479
Avg. Drawdown [%]                    -1.94628
Max. Drawdown Duration       90 days 00:00:00
Avg. Drawdown Duration       19 days 00:00:00
# Trades                          

Giving control to LLM for "best" startegy

In [17]:
class RsiWmaBollingerStrategy(Strategy):
    rsi_period = 10
    rsi_buy = 28
    rsi_sell = 60
    bb_period = 15
    bb_std = 1.5
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        close = self.data.Close

        # Use TA-Lib for all indicators (multiprocessing safe)
        self.rsi = talib.RSI(close, timeperiod=self.rsi_period)
        self.wma = talib.WMA(close, timeperiod=self.bb_period)
        self.std = pd.Series(close).rolling(self.bb_period).std().values

        self.bb_upper = self.wma + self.bb_std * self.std
        self.bb_lower = self.wma - self.bb_std * self.std

    def next(self):
        i = len(self.data.Close) - 1
        price = self.data.Close[i]

        # Handle edge cases
        if np.isnan(self.rsi[i]) or np.isnan(self.bb_lower[i]) or np.isnan(self.bb_upper[i]):
            return

        if self.rsi[i] < self.rsi_buy and price < self.bb_lower[i]:
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        elif self.position:
            if self.rsi[i] > self.rsi_sell or price > self.bb_upper[i]:
                self.position.close()



# Fetch data
df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run backtest
bt = Backtest(df, RsiWmaBollingerStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
#bt.plot(filename="RsiWmaBollingerStrategy.html")


Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                     2.99625
Equity Final [$]                   9817.83811
Equity Peak [$]                   10202.94688
Commissions [$]                     114.40817
Return [%]                           -1.82162
Buy & Hold Return [%]                56.27101
Return (Ann.) [%]                    -0.86381
Volatility (Ann.) [%]                 4.10005
CAGR [%]                             -0.59447
Sharpe Ratio                         -0.21068
Sortino Ratio                        -0.24492
Calmar Ratio                         -0.11863
Alpha [%]                            -3.25188
Beta                                  0.02542
Max. Drawdown [%]                    -7.28145
Avg. Drawdown [%]                    -7.28145
Max. Drawdown Duration      329 days 00:00:00
Avg. Drawdown Duration      329 days 00:00:00
# Trades                          

At this point, we have few strategies which works good for some and not so good for others. So let's filter/screen positive stock for us

In [18]:
def run_multi_stock_backtest(tickers,  period="60d"):
    results = []

    for ticker in tickers:
        time.sleep(2.3)
        print(f"Running backtest for {ticker}...")
        try:
            df = get_stock_data(ticker, interval="1d", start="2023-02-01", end="2025-03-20")

            if df.empty:
                continue

            bt = Backtest(df, RsiWmaBollingerStrategy, cash=10_000, commission=0.002)     
            stats = bt.run()

            results.append({
                "Ticker": ticker,
                "Return [%]": stats['Return [%]'],
                "Win Rate [%]": stats['Win Rate [%]'],
                "Sharpe Ratio": stats.get('Sharpe Ratio', None),
                "Trades": stats['# Trades'],
                "Avg Trade Duration": stats['Avg. Trade Duration'],
                "Exposure Time [%]": stats['Exposure Time [%]'],
            })

        except Exception as e:
            print(f"Error with {ticker}: {e}")

    df_results = pd.DataFrame(results)
    #df_results.sort_values(by="Return [%]", ascending=False, inplace=True)
    return df_results

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NFLX", "META", "NVDA", "AMD", "INTC"]
results = run_multi_stock_backtest(tickers,  period="60d")
print(results)


Running backtest for AAPL...


Backtest.run:   0%|          | 0/533 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=130: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=275: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Running backtest for MSFT...


Running backtest for GOOGL...


Backtest.run:   0%|          | 0/533 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=187: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


...

Running backtest for META...


Running backtest for NVDA...


Running backtest for AMD...


Running backtest for INTC...


  Ticker  Return [%]  Win Rate [%]  Sharpe Ratio  Trades Avg Trade Duration  \
0   AAPL    1.925269     30.000000      0.164315      10             5 days   
1   MSFT   -1.821619     33.333333     -0.210684       3             6 days   
2  GOOGL   -5.002818     42.857143     -0.285235       7             7 days   
3   AMZN   -1.333607     25.000000     -0.219513       4             2 days   
4   TSLA    0.214620     23.076923      0.009523      13             1 days   
5   NFLX    6.715005     40.000000      0.422290      10             3 days   
6   META    8.357799     60.000000      0.883693       5             2 days   
7   NVDA    0.000000           NaN           NaN       0                NaT   
8    AMD   -1.567559     50.000000     -0.138799       4             3 days   
9   INTC  -12.726704      9.090909     -0.861694      11             2 days   

   Exposure Time [%]  
0           7.490637  
1           2.996255  
2           6.741573  
3           1.685393  
4           3.5

## 3. Integrating a black-box indicator (e.g., ML model)
Now we will add a black-box function which can be an ML model which can provide BUY or SELL signal. We will pass the dataframe containing the features and function can use this to predict the position. This then can be used in `next` function.

In [19]:
def dummy_ml_model(df):
    """
    Simulated ML model that predicts BUY signals. We don't consider SELL (short positions).
    :param df: DataFrame of last `N` rows (OHLCV features).
    :return: 1 if BUY signal, else 0.
    """
    # Simulated logic: Buy if the last row closes lower than the first row in the window
    if df.iloc[-1]['Close'] < df.iloc[0]['Close']:  
        return 1  # BUY Signal
    return 0  # No action

class MLTradingStrategy(Strategy):
    feature_window = 5 # window size for features 
    stop_loss_pct = 0.02 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            "Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = dummy_ml_model(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
#bt.plot(filename="ml__strategy.html")

Backtest.run:   0%|          | 0/533 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=15: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=16: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=17: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=18: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=20: Bro

Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                    67.79026
Equity Final [$]                   7589.70385
Equity Peak [$]                   11215.16409
Commissions [$]                    2703.35984
Return [%]                          -24.10296
Buy & Hold Return [%]                56.27101
Return (Ann.) [%]                   -12.20356
Volatility (Ann.) [%]                16.00679
CAGR [%]                             -8.55626
Sharpe Ratio                          -0.7624
Sortino Ratio                        -0.86535
Calmar Ratio                         -0.36042
Alpha [%]                           -54.51215
Beta                                  0.54041
Max. Drawdown [%]                   -33.85961
Avg. Drawdown [%]                    -8.31362
Max. Drawdown Duration      342 days 00:00:00
Avg. Drawdown Duration       82 days 00:00:00
# Trades                          

## 4. Integrating ESN model from `1_esn_le2e.ipynb`
Here, our function will look for some threshold above which we can have BUY.

In [21]:
from utils import StockPricePredictor

predictor = StockPricePredictor(model_path="stock_esn_model_7.pkl", 
                                scaler_path="scaler_esn_model_7.pkl")

def if_buy_signal_with_esn(df, threshold=0.02):
    """
    Determines if a BUY signal should be generated based on the ESN model's prediction.
    
    Args:
        df (DataFrame): The dataframe containing the latest stock data.
        threshold (float): The minimum percentage increase required to trigger a BUY signal.
    
    Returns:
        int: 1 if BUY signal is triggered, otherwise 0.
    """
    predicted_close = predictor.predict(df)
    last_close = df.iloc[-1]['Close']
    
    # Check if predicted close is at least (1 + threshold)% higher than last close
    if predicted_close >= last_close * (1 + threshold):
        return 1  # BUY Signal
    
    return 0  # No action


class MLTradingStrategy(Strategy):
    feature_window = 7 # window size for features 
    stop_loss_pct = 0.025 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            #"Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = if_buy_signal_with_esn(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# bt.plot(filename="esn_strategy.html")

Backtest.run:   0%|          | 0/533 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=10: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=18: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                                

Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                      8.2397
Equity Final [$]                  10031.94359
Equity Peak [$]                     10487.459
Commissions [$]                     272.76286
Return [%]                            0.31944
Buy & Hold Return [%]                56.27101
Return (Ann.) [%]                     0.15062
Volatility (Ann.) [%]                 6.03431
CAGR [%]                              0.10349
Sharpe Ratio                          0.02496
Sortino Ratio                         0.03706
Calmar Ratio                          0.01509
Alpha [%]                            -3.72485
Beta                                  0.07187
Max. Drawdown [%]                    -9.98271
Avg. Drawdown [%]                    -5.26252
Max. Drawdown Duration       52 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                          

# Predicting for today

In [22]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
df = get_stock_data(ticker="MSFT", interval="1d", start="2025-03-20", end=today)
df = df.drop(columns=['Volume'])
df.head(10)

Price,Open,High,Low,Close
Date,,,,
2025-03-20,385.739990,391.790009,383.279999,386.839996
2025-03-21,383.220001,391.739990,382.799988,391.260010
2025-03-24,395.399994,395.399994,389.809998,393.079987
2025-03-25,393.920013,396.359985,392.640015,395.160004
2025-03-26,395.000000,395.309998,388.570007,389.970001
2025-03-27,390.130005,392.239990,387.399994,390.579987
2025-03-28,388.079987,389.130005,376.929993,378.799988
2025-03-31,372.540009,377.070007,367.239990,375.390015
2025-04-01,374.649994,382.850006,373.230011,382.190002


In [ ]:
signal = if_buy_signal_with_esn(df.tail(7)) 
print("signal", signal)
predicted_close = predictor.predict(df.tail(7))
print("predicted_close", predicted_close) 

signal 0
predicted_close 380.3441784198214


END OF NOTEBOOK